In [16]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

Data Preprocessing

In [17]:
# movies - index, movie name, catagory
movies = pd.read_csv('ml-1m/movies.dat', sep='::', header=None, engine='python', encoding='latin-1')
# users - index, gender, age, financial info, zip code
users = pd.read_csv('ml-1m/users.dat', sep='::', header=None, engine='python', encoding='latin-1')
# ratings - index, user id, movie id, rating (1-5), timestaps (not required)
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1')
# training set - index, user index, movie index, rating (1-5), time stamp (not required)
training_set = pd.read_csv('ml-100k/u1.base', delimiter='\t')
training_set = np.array(training_set, dtype='int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter='\t')
test_set = np.array(test_set, dtype='int')

# Getting the number of users and movies
# taking max of max as the value can be in either train / test set
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))

# converting the data into an array with users in lines and movies in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:, 1][data[:, 0] == id_users]
        id_ratings = data[:, 2][data[:, 0] == id_users]
        ratings = np.zeros(nb_movies) # this also adds a 0 for all the movies not rated by the user
        ratings[id_movies - 1] = id_ratings 
        new_data.append(list(ratings)) # list of lists
    return new_data

training_set = convert(training_set)
test_set = convert(test_set)

# converting the data into torch tensors
# floatTensor class expects a list of lists
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

# Converting ratings into binary ratings, 1 (liked) & 0 (Not liked)
training_set[training_set == 0] = -1 # these were the movies not rated by the user.
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0 # torch doesn't accept 'or' operator, hence they have been written separately
training_set[training_set >= 3] = 1

test_set[test_set == 0] = -1 # these were the movies not rated by the user.
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0 # torch doesn't accept 'or' operator, hence they have been written separately
test_set[test_set >= 3] = 1

RBM Implementation

In [ ]:
# RBM is a probabilistic graphical model
# creatingg architecture of Neural Network 
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv) # weights are initialized as torch tensor with values based on normal distribution.
        self.a = torch.randn(1, nh) # 1 - corresponds to the batch, nh - for bias for hidden nodes
        self.b = torch.randn(1, nv) # bias for visible nodes

    def sample_h(self, x): # x - visble node
        wx = torch.mm(x, self.W.t()) # function torch.mm multiplies two torch tensors
        activation = wx + self.a.expand_as(wx)
        # probability a hidden node is activated given the value of a visible node
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v) # returns sampling of the hidden node as well
    
    def sample_v(self, y): # y - hidden node
        wy = torch.mm(y, self.W) # function torch.mm multiplies two torch tensors
        activation = wy + self.b.expand_as(wy)
        # probability a visible node is activated given the value of a hidden node
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h) # returns sampling of the visible node as well
    
    # Τhis function gives error
    # def train(self, v0, vk, ph0, phk): # contrastive diveregence with gibbs sampling
    #     self.W += torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)
    #     self.b += torch.sum((v0 - vk), 0) # that 0 is brought in to keep self.b as a 2D tensor
    #     self.a += torch.sum((ph0 - phk), 0)

    # This function is the corrected one, transpose is applied to ph0 and phk instead of v0 and vk
    def train(self, v0, vk, ph0, phk): # contrastive divergence with Gibbs sampling
        # Correcting the dimension mismatch by transposing hidden units
        self.W += torch.mm(ph0.t(), v0) - torch.mm(phk.t(), vk)
        self.b += torch.sum((v0 - vk), 0)  # Updating biases for visible units
        self.a += torch.sum((ph0 - phk), 0)  # Updating biases for hidden units


In [23]:
nv = len(training_set[0])
nh = 100 # aimed to detect 100 features
batch_size = 100 
rbm = RBM(nv, nh)

Training the RBM

In [24]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user: id_user + batch_size]
        v0 = training_set[id_user: id_user + batch_size]
        ph0, _ = rbm.sample_h(v0)

        for k in range(10):
            _, hk = rbm.sample_h(vk)
            _, vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0] # ensuring the model is not trained on data that doesn't exist, these were given -1 value earlier
        phk, _ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>0] - vk[v0>0])) # difference between actual rating and prediction (for ratings that exist)
        s += 1
    print('Epoch: '+str(epoch)+' loss: ', str(train_loss/s)) # loss is normalized while printing
        


Epoch: 1 loss:  tensor(0.2739)
Epoch: 2 loss:  tensor(0.1520)
Epoch: 3 loss:  tensor(0.1492)
Epoch: 4 loss:  tensor(0.1546)
Epoch: 5 loss:  tensor(0.1485)
Epoch: 6 loss:  tensor(0.1466)
Epoch: 7 loss:  tensor(0.1507)
Epoch: 8 loss:  tensor(0.1489)
Epoch: 9 loss:  tensor(0.1475)
Epoch: 10 loss:  tensor(0.1483)


Testing the RBM

In [36]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user: id_user + 1]
    vt = test_set[id_user: id_user + 1]
    if len(vt[vt >= 0]) > 0:
        _, h = rbm.sample_h(v)
        _, v = rbm.sample_v(h)
        # Only compare entries where vt >= 0
        mask = vt >= 0  # Create a mask for valid entries
        test_loss += torch.mean(torch.abs(vt[mask] - v[mask]))  # Use the mask for comparison
        s += 1
print('test loss: ', str(test_loss / s))  # Loss is normalized while printing


test loss:  tensor(0.2409)
